## Save middle slices of scans as images
Most image models expect a three-channel input image. Will take the middle three grayscale slices, and dilated middle three slices centered on the middle, and save them as three-channel images.

In [1]:
import numpy as np
from PIL import Image
import os

In [2]:
! tree -d ..

..
├── data
│   ├── __MACOSX
│   │   └── MRNet-v1.0
│   │       ├── train
│   │       │   ├── axial
│   │       │   ├── coronal
│   │       │   └── sagittal
│   │       └── valid
│   ├── mid1
│   │   ├── train
│   │   │   ├── axial
│   │   │   ├── coronal
│   │   │   └── sagittal
│   │   └── valid
│   │       ├── axial
│   │       ├── coronal
│   │       └── sagittal
│   ├── MRNet-small
│   │   ├── train
│   │   │   ├── axial
│   │   │   ├── coronal
│   │   │   └── sagittal
│   │   └── valid
│   │       ├── axial
│   │       ├── coronal
│   │       └── sagittal
│   └── MRNet-v1.0
│       ├── train
│       │   ├── axial
│       │   ├── coronal
│       │   └── sagittal
│       └── valid
│           ├── axial
│           ├── coronal
│           └── sagittal
└── mrnet-fastai
    └── exp

37 directories


Copy data to another directory, for example `/mid3d0`, for middle 3 slices, 0 dilation.
Do this for d in (0,1,2) and perhaps greater than 2.

In [3]:
!cp -r ../data/MRNet-v1.0 ../data/mid3d0

Recurse through /train and /valid directories, load the scan file (.npy), select middle 3 slices, possibly with gaps of size dilation between them, and save as 3-channel images. Discard the copied files.

In [7]:
dilations = [0,1,2]
for d in dilations:
    print('-----------------------')
    print('Creating png files centered on middle slices with dilations of size {}'.format(d))
    for ds in ('train','valid'):
        for p in ('axial','coronal','sagittal'):
            dirpath = '../data/mid3d{}/{}/{}'.format(d, ds, p)
            npy_files = [f for f in os.listdir(dirpath) if f[-4:]=='.npy']
            print('Processing {} .npy files in {}'.format(len(npy_files), dirpath))

            for npyf in npy_files:
                npyfilepath = dirpath + '/' + npyf
                #print('Converting {}'.format(npyfilepath))
                # load the array
                scanarray = np.load(npyfilepath)
                #print('Number of slices for this scan: {}'.format(scanarray.shape[0]))
                # get the middle slice index
                midslice = scanarray.shape[0]//2
                # get indices for three slices centered on midslice, skipping dilation d images between the slices
                slice_indices = range(midslice-(1+d),midslice+(1+d+1),1+d)
                slices3  = scanarray[slice_indices,:,:]
                # put channels last for PIL
                slices3  = np.moveaxis(slices3, 0, -1)
                # convert to rgb
                im = Image.fromarray(slices3)
                # save file
                # replace .npy with .png
                pngf = npyf[:-4] + '.png'
                pngfilepath = dirpath + '/' + pngf
                im.save(pngfilepath)
                # remove corresponding .npy file
                os.remove(npyfilepath)

            print('===============================')        

-----------------------
Creating png files centered on middle slices with dilations of size 2
Processing 1130 .npy files in ../data/mid3d2/train/axial
Processing 1130 .npy files in ../data/mid3d2/train/coronal
Processing 1130 .npy files in ../data/mid3d2/train/sagittal
Processing 120 .npy files in ../data/mid3d2/valid/axial
Processing 120 .npy files in ../data/mid3d2/valid/coronal
Processing 120 .npy files in ../data/mid3d2/valid/sagittal


## Take just middle slice from each scan

In [11]:
dilation = ''
for d in ('train','valid'):
    for p in ('axial','coronal','sagittal'):
        dirpath = '../data/mid1{}/{}/{}'.format(dilation, d, p)
        npy_files = [f for f in os.listdir(dirpath) if f[-4:]=='.npy']
        print('Processing {} .npy files in {}'.format(len(npy_files), dirpath))

        for f in npy_files:
            filepath = dirpath + '/' + f
            #print('Converting {}'.format(filepath))
            # load the array
            scanarray = np.load(filepath)
            # calculate interpolation factor(s)
            #print('Number of slices for this scan: {}'.format(scanarray.shape[0]))
            midslice = scanarray.shape[0]//2
            im = Image.fromarray(scanarray[midslice,:,:], 'L').convert('RGB')
            # replace .npy with .png
            newf = f[:-4] + '.png'
            newfilepath = dirpath + '/' + newf
            im.save(newfilepath)
            # remove corresponding .npy file
            os.remove(filepath)
        
        print('===============================')        

Found 1130 .npy files in ../data/mid1/train/axial
Found 1130 .npy files in ../data/mid1/train/coronal
Found 1130 .npy files in ../data/mid1/train/sagittal
Found 120 .npy files in ../data/mid1/valid/axial
Found 120 .npy files in ../data/mid1/valid/coronal
Found 120 .npy files in ../data/mid1/valid/sagittal
